## Creating a Classification Model

In this exercise, you will implement a classification model that uses features of a flight to predict whether or not it will be late.

### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler

### Load Source Data
The data for this exercise is provided as a CSV file containing details of flights that has already been cleaned up for modeling. The data includes specific characteristics (or *features*) for each flight, as well as a *label* column indicating whether or not the flight was late (a flight with an arrival delay of more than 25 minutes is considered *late*).

You will load this data into a dataframe and display it.

In [4]:
path="wasbs://blob-for-demo-data@juliansblobstore.blob.core.windows.net/"

flightSchema = StructType([
  StructField("DayofMonth", IntegerType(), False),
  StructField("DayOfWeek", IntegerType(), False),
  StructField("Carrier", StringType(), False),
  StructField("OriginAirportID", IntegerType(), False),
  StructField("DestAirportID", IntegerType(), False),
  StructField("DepDelay", IntegerType(), False),
  StructField("ArrDelay", IntegerType(), False),
  StructField("Late", IntegerType(), False),
])


data = spark.read.csv(path+"flights.csv", schema=flightSchema, header=True)
data.show()

+----------+---------+-------+---------------+-------------+--------+--------+----+
DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|Late|
+----------+---------+-------+---------------+-------------+--------+--------+----+
 21| 2| WN| 10721| 13342| 26| 57| 1|
 13| 1| AA| 15016| 12892| 51| 27| 1|
 5| 5| FL| 10397| 11433| 9| 4| 0|
 22| 1| US| 11278| 14100| 35| 71| 1|
 23| 4| WN| 12451| 10693| 9| 5| 0|
 5| 7| AA| 11298| 15016| 39| 42| 1|
 4| 6| UA| 13930| 14307| 71| 58| 1|
 10| 3| 9E| 14307| 11433| 68| 140| 1|
 29| 7| UA| 14057| 14771| 130| 125| 1|
 14| 7| UA| 14771| 11292| 20| 42| 1|
 14| 6| UA| 12892| 11618| 6| -21| 0|
 16| 3| AA| 11298| 13204| 5| -7| 0|
 7| 3| VX| 14747| 14771| 143| 129| 1|
 20| 2| DL| 13204| 12478| 9| 5| 0|
 28| 5| DL| 11292| 13487| 1| -3| 0|
 17| 6| MQ| 11042| 13303| 12| 26| 1|
 3| 3| EV| 12266| 15016| 20| 3| 0|
 2| 4| DL| 10397| 13198| 32| 23| 0|
 4| 5| WN| 12889| 13342| 90| 80| 1|
 25| 6| UA| 10721| 12266| -5| -27| 0|
+----------+---------+-------+---------------+-------------+--------+--------+----+
only showing top 20 rows

In [5]:
display(data.head(20))

DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay,Late
21,2,WN,10721,13342,26,57,1
13,1,AA,15016,12892,51,27,1
5,5,FL,10397,11433,9,4,0
22,1,US,11278,14100,35,71,1
23,4,WN,12451,10693,9,5,0
5,7,AA,11298,15016,39,42,1
4,6,UA,13930,14307,71,58,1
10,3,9E,14307,11433,68,140,1
29,7,UA,14057,14771,130,125,1
14,7,UA,14771,11292,20,42,1


### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing.

In [7]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print ("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 445731 Testing Rows: 191352

### Prepare the Training Data
To train the classification model, you need a training data set that includes a vector of numeric features, and a label column. In this exercise, you will use the **StringIndexer** class to generate a numeric category for each discrete **Carrier** string value, and then use the **VectorAssembler** class to transform the numeric features that would be available for a flight that hasn't yet arrived into a vector, and then rename the **Late** column to **label** as this is what we're going to try to predict.

*Note: This is a deliberately simple example. In reality you'd likely perform mulitple data preparation steps, and later in this course we'll examine how to encapsulate these steps in to a pipeline. For now, we'll just use the numeric features as they are to define the training dataset.*

In [9]:
# Carrier is a string, and we need our features to be numeric - so we'll generate a numeric index for each distinct carrier string, and transform the dataframe to add that as a column
carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
numTrain = carrierIndexer.fit(train).transform(train)

# Now we'll assemble a vector of all the numeric feature columns (other than ArrDelay, which we wouldn't have for enroute flights)
assembler = VectorAssembler(inputCols = ["DayofMonth", "DayOfWeek", "CarrierIdx", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")
training = assembler.transform(numTrain).select(col("features"), col("Late").alias("label"))
training.show()

+--------------------+-----+
 features|label|
+--------------------+-----+
[1.0,1.0,10.0,103...| 1|
[1.0,1.0,10.0,105...| 1|
[1.0,1.0,10.0,107...| 0|
[1.0,1.0,10.0,107...| 1|
[1.0,1.0,10.0,107...| 0|
[1.0,1.0,10.0,110...| 0|
[1.0,1.0,10.0,110...| 1|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 0|
[1.0,1.0,10.0,111...| 0|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 0|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,112...| 0|
[1.0,1.0,10.0,114...| 0|
[1.0,1.0,10.0,114...| 1|
[1.0,1.0,10.0,114...| 1|
+--------------------+-----+
only showing top 20 rows

### Train a Classification Model
Next, you need to train a classification model using the training data. To do this, create an instance of the classification algorithm you want to use and use its **fit** method to train a model based on the training dataframe. In this exercise, you will use a *Logistic Regression* classification algorithm - but you can use the same technique for any of the classification algorithms supported in the spark.ml API.

In [11]:
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
model = lr.fit(training)
print ("Model trained!")

Model trained!

In [12]:
model.save

Out[12]: LogisticRegressionModel: uid = LogisticRegression_68cad38e9e49, numClasses = 2, numFeatures = 6

### Prepare the Testing Data
Now that you have a trained model, you can test it using the testing data you reserved previously. First, you need to prepare the testing data in the same way as you did the training data by transforming the feature columns into a vector. This time you'll rename the **Late** column to **trueLabel**.

In [14]:
# Transform the test data to add the numeric carrier index
numTest = carrierIndexer.fit(test).transform(test)

# Generate the features vector and label
testing = assembler.transform(numTest).select(col("features"), col("Late").alias("trueLabel"))
testing.show()

+--------------------+---------+
 features|trueLabel|
+--------------------+---------+
[1.0,1.0,10.0,103...| 0|
[1.0,1.0,10.0,107...| 1|
[1.0,1.0,10.0,110...| 0|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 0|
[1.0,1.0,10.0,111...| 0|
[1.0,1.0,10.0,112...| 1|
[1.0,1.0,10.0,124...| 1|
[1.0,1.0,10.0,124...| 1|
[1.0,1.0,10.0,124...| 1|
[1.0,1.0,10.0,132...| 0|
[1.0,1.0,10.0,134...| 1|
[1.0,1.0,10.0,134...| 1|
[1.0,1.0,10.0,139...| 1|
[1.0,1.0,10.0,153...| 0|
[1.0,1.0,2.0,1039...| 1|
[1.0,1.0,2.0,1069...| 1|
[1.0,1.0,2.0,1072...| 0|
[1.0,1.0,2.0,1082...| 0|
+--------------------+---------+
only showing top 20 rows

### Test the Model
Now you're ready to use the **transform** method of the model to generate some predictions. You can use this approach to predict delay status for flights where the label is unknown; but in this case you are using the test data which includes a known true label value, so you can compare the predicted status to the actual status.

In [16]:
prediction = model.transform(testing)
predicted = prediction.select("features", "probability", col("prediction").cast("Int"), "trueLabel")
predicted.show(100, truncate=False)

+------------------------------------+----------------------------------------+----------+---------+
features |probability |prediction|trueLabel|
+------------------------------------+----------------------------------------+----------+---------+
[1.0,1.0,10.0,10397.0,13851.0,2.0] |[0.5659091885584878,0.4340908114415122] |0 |0 |
[1.0,1.0,10.0,10721.0,12478.0,37.0] |[0.40524094535798255,0.5947590546420175]|1 |1 |
[1.0,1.0,10.0,11066.0,13487.0,-1.0] |[0.5821052828254059,0.41789471717459414]|0 |0 |
[1.0,1.0,10.0,11193.0,10529.0,16.0] |[0.5004694342208846,0.49953056577911537]|0 |1 |
[1.0,1.0,10.0,11193.0,11298.0,55.0] |[0.3286501518932936,0.6713498481067064] |1 |1 |
[1.0,1.0,10.0,11193.0,13244.0,-1.0] |[0.582349047369532,0.417650952630468] |0 |0 |
[1.0,1.0,10.0,11193.0,13342.0,-3.0] |[0.5914537737419076,0.4085462262580925] |0 |0 |
[1.0,1.0,10.0,11278.0,12478.0,58.0] |[0.31843195230730525,0.6815680476926947]|1 |1 |
[1.0,1.0,10.0,12478.0,10821.0,44.0] |[0.38014820632582985,0.6198517936741702]|1 |1 |
[1.0,1.0,10.0,12478.0,12264.0,129.0]|[0.1139296537077936,0.8860703462922064] |1 |1 |
[1.0,1.0,10.0,12478.0,13930.0,-1.0] |[0.5894036538386714,0.4105963461613286] |0 |1 |
[1.0,1.0,10.0,13244.0,10423.0,-6.0] |[0.6100835792062347,0.3899164207937654] |0 |0 |
[1.0,1.0,10.0,13487.0,10821.0,90.0] |[0.21089953157099797,0.789100468429002] |1 |1 |
[1.0,1.0,10.0,13487.0,14683.0,65.0] |[0.3027574989105265,0.6972425010894735] |1 |1 |
[1.0,1.0,10.0,13931.0,12478.0,62.0] |[0.3135847242803134,0.6864152757196866] |1 |1 |
[1.0,1.0,10.0,15304.0,14492.0,-9.0] |[0.6382100354153305,0.3617899645846695] |0 |0 |
[1.0,1.0,2.0,10397.0,13303.0,37.0] |[0.408512986861155,0.591487013138845] |1 |1 |
[1.0,1.0,2.0,10693.0,12892.0,-1.0] |[0.583091697680457,0.41690830231954307] |0 |1 |
[1.0,1.0,2.0,10721.0,13303.0,-8.0] |[0.6148799512385721,0.3851200487614279] |0 |0 |
[1.0,1.0,2.0,10821.0,13303.0,0.0] |[0.5798008222087951,0.42019917779120486]|0 |0 |
[1.0,1.0,2.0,11042.0,11298.0,-1.0] |[0.5824051375351581,0.41759486246484184]|0 |0 |
[1.0,1.0,2.0,11278.0,13303.0,0.0] |[0.5819607216967708,0.4180392783032291] |0 |1 |
[1.0,1.0,2.0,11278.0,13930.0,74.0] |[0.2624877718159913,0.7375122281840086] |1 |1 |
[1.0,1.0,2.0,11298.0,10140.0,6.0] |[0.5501497654888086,0.44985023451119144]|0 |0 |
[1.0,1.0,2.0,11298.0,10397.0,10.0] |[0.5321791362304782,0.46782086376952187]|0 |0 |
[1.0,1.0,2.0,11298.0,10397.0,19.0] |[0.49063986192765224,0.5093601380723477]|1 |0 |
[1.0,1.0,2.0,11298.0,10423.0,-3.0] |[0.5912945647064795,0.40870543529352066]|0 |0 |
[1.0,1.0,2.0,11298.0,11278.0,-2.0] |[0.5880684280711035,0.4119315719288966] |0 |0 |
[1.0,1.0,2.0,11298.0,11278.0,43.0] |[0.3832651657592585,0.6167348342407415] |1 |1 |
[1.0,1.0,2.0,11298.0,11618.0,31.0] |[0.4373611909180221,0.5626388090819778] |1 |1 |
[1.0,1.0,2.0,11298.0,12892.0,-2.0] |[0.5904208044542727,0.40957919554572725]|0 |0 |
[1.0,1.0,2.0,11298.0,13851.0,2.0] |[0.5738438271739198,0.4261561728260802] |0 |0 |
[1.0,1.0,2.0,11298.0,13851.0,44.0] |[0.3825591005191076,0.6174408994808924] |1 |1 |
[1.0,1.0,2.0,11298.0,13891.0,65.0] |[0.2959653616127438,0.7040346383872562] |1 |1 |
[1.0,1.0,2.0,11298.0,13930.0,27.0] |[0.4590848429828662,0.5409151570171337] |1 |0 |
[1.0,1.0,2.0,11298.0,14122.0,26.0] |[0.4639653230067792,0.5360346769932208] |1 |1 |
[1.0,1.0,2.0,11298.0,14683.0,22.0] |[0.4832357625387578,0.5167642374612421] |1 |0 |
[1.0,1.0,2.0,11433.0,11298.0,85.0] |[0.22283475988834528,0.7771652401116547]|1 |1 |
[1.0,1.0,2.0,12264.0,12892.0,0.0] |[0.5860097157797383,0.4139902842202617] |0 |0 |
[1.0,1.0,2.0,12478.0,13303.0,-7.0] |[0.6185736477078555,0.3814263522921445] |0 |0 |
[1.0,1.0,2.0,12478.0,13303.0,-1.0] |[0.5920883819800422,0.40791161801995784]|0 |0 |
[1.0,1.0,2.0,12889.0,11298.0,21.0] |[0.49047677313558163,0.5095232268644183]|1 |0 |
[1.0,1.0,2.0,12892.0,13930.0,2.0] |[0.5815091069545167,0.41849089304548326]|0 |0 |
[1.0,1.0,2.0,12892.0,14771.0,-4.0] |[0.6094328631471895,0.3905671368528105] |0 |0 |
[1.0,1.0,2.0,12892.0,14771.0,63.0] |[0.3114439765562

Looking at the result, the **prediction** column contains the predicted value for the label, and the **trueLabel** column contains the actual known value from the testing data. The **probability** column shows the probability score for each class (0 or 1). It looks like there are a mix of correct and incorrect predictions, and the ones that are incorrect tend to have fairly close probabilities for each class. Later in this course you'll learn how to measure the accuracy of a model.